In [31]:
%load_ext autoreload

import xarray as xr
import rasterio
import pandas as pd
import numpy as np
import osr
import os
import arcpy
import datetime

sys.path.append(r'C:\Users\Lewis\Documents\GitHub\tenement-tools\shared')
import arc, cog, satfetcher, tools

sys.path.append(r'C:\Users\Lewis\Documents\GitHub\tenement-tools\modules')
import gdvspectra

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
#ds_thresh = xr.open_dataset(r"C:\Users\Lewis\Desktop\working\thresh.nc")

In [68]:
arcpy.env.addOutputsToMap = False
mdr = arcpy.CopyRaster_management(in_raster=r"C:\Users\Lewis\Desktop\working\thresh.nc", 
                                  out_rasterdataset='mdr_thresh.crf')

In [69]:
mdr_filter = arcpy.sa.MajorityFilter(mdr, "FOUR", "MAJORITY")
mdr_filter.save('in_memory/mdr_thresh_filter.crf')

In [70]:
mdr_filter_binary = arcpy.sa.Con(mdr_filter, 1, None, "Value = 1")
mdr_filter_binary.save('in_memory/mdr_thresh_filter_binary.crf')

In [71]:
mdr_filter = arcpy.CopyRaster_management(in_raster='in_memory/mdr_thresh_filter_binary.crf', 
                                  out_rasterdataset='mdr_thresh_filter.crf')

In [72]:
# todo make this safe
aprx = arcpy.mp.ArcGISProject('CURRENT')
m = aprx.activeMap
m.addDataFromPath(mdr_filter) 

In [89]:
# apply cmap
arcpy.env.addOutputsToMap = True
lyr = arc.apply_cmap(aprx=aprx, 
                     #lyr_name='mdr_thresh.crf', 
                     lyr_name='mdr_thresh_filter.crf', 
                     cmap_name='Set 2 (3 Classes)', 
                     cutoff_pct=0.01)

# set alpha
lyr.transparency = 25